In [0]:
import pandas as pd
import numpy as np
import datetime
import math

In [2]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/My\ Drive

 2018_schedule.csv      gamelogs	 'Shan&Des'   通訊錄20120504.gsheet
 2018_schedule.gsheet   IoT_and_ML_2019   履歷
'Colab Notebooks'       map		  論文


In [0]:
fpbase = "/content/drive/My Drive/gamelogs/"
recentGames = 7

In [0]:
def monthToNum(x):
    return {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }.get(x, None)

def parseTimeString(timeStr):
    x = timeStr.split()
    return "2018-{0:s}-{1:2d}".format(monthToNum(x[0]), int(x[1]))

In [0]:
def getRecentOPS(gameDate, playerID):
    fname = '2018_b_{}.csv'.format(playerID)
    tempDate = '2018-03-28'
    tdelta = datetime.timedelta(days=1)

    batting_df = pd.read_csv(fpbase+fname)
    for index, row in batting_df.iterrows():
        #FIX-ME: fix the date format in csv is better solution
        if type(row['Date']) is float:
            dl = tempDate.split('-')
            d = datetime.date(int(dl[0]), int(dl[1]), int(dl[2]))
            d = d + tdelta
            tempDate = d.isoformat()
            batting_df.at[index, 'Date'] = tempDate
        else:
            tempDate = parseTimeString(row['Date'])
            batting_df.at[index, 'Date'] = tempDate

    gamesBeforeDate = batting_df \
                      .loc[batting_df['Date'] < gameDate]
    if gamesBeforeDate.empty:
        return 0

    recentStat_df = gamesBeforeDate \
                    .sort_values(by='Date', ascending=False) \
                    .head(recentGames)

    #OPB = (H + BB + HBP)/(AB + BB + HBP + SF)
    OBP = (recentStat_df['H'].sum() \
        + recentStat_df['BB'].sum() \
        + recentStat_df['HBP'].sum()) / \
        (recentStat_df['AB'].sum() \
            + recentStat_df['BB'].sum() \
            + recentStat_df['HBP'].sum() \
            +recentStat_df['SF'].sum())
    OBP = round(OBP, 3)

    #SLG = (H + 2B*2 + 3B*3 + HR*4)/AB
    SLG = (recentStat_df['H'].sum() \
        + 2 * recentStat_df['2B'].sum() \
        + 3 * recentStat_df['3B'].sum() \
        + 4 * recentStat_df['HR'].sum()) / \
        recentStat_df['AB'].sum()
    SLG = round(SLG, 3)

    OPS = round((OBP + SLG), 3)
    print(type(OPS))
    if math.isnan(OPS):
        OPS = 0
    #print(OBP)
    #print(SLG)
    #print(OPS)
    return OPS

In [158]:
playerID = "lambja01"
gdate = '2018-03-30'
getRecentOPS(gdate, playerID)

<class 'numpy.float64'>


1.2

In [0]:
def getRecentWHIP(gameDate, playerID):
    fname = '2018_p_{}.csv'.format(playerID)
    tempDate = '2018-03-28'
    tdelta = datetime.timedelta(days=1)

    pitching_df = pd.read_csv(fpbase+fname)
    for index, row in pitching_df.iterrows():
        #FIX-ME: fix the date format in csv is better solution
        if type(row['Date']) is float:
            dl = tempDate.split('-')
            d = datetime.date(int(dl[0]), int(dl[1]), int(dl[2]))
            d = d + tdelta
            tempDate = d.isoformat()
            pitching_df.at[index, 'Date'] = tempDate
        else:
            tempDate = parseTimeString(row['Date'])
            pitching_df.at[index, 'Date'] = tempDate

    gamesBeforeDate = pitching_df \
                      .loc[pitching_df['Date'] < gameDate]
    if gamesBeforeDate.empty:
        return 0

    recentStat_df = gamesBeforeDate \
                    .sort_values(by='Date', ascending=False) \
                    .head(recentGames)

    #WHIP = (H + BB)/IP
    WHIP = (recentStat_df['H'].sum() + recentStat_df['BB'].sum()) / \
            recentStat_df['IP'].sum()

    WHIP = round(WHIP, 3)
    #print(WHIP)
    return WHIP

In [78]:
#playerID = "grayjo02"
#gdate = '2018-09-30'
#getRecentWHIP(gdate, playerID)

1.699

In [0]:
awayLineup = ['A_b1', 'A_b2', 'A_b3', 'A_b4', 'A_b5',
              'A_b6', 'A_b7', 'A_b8', 'A_b9']
homeLineup = ['H_b1', 'H_b2', 'H_b3', 'H_b4', 'H_b5',
              'H_b6', 'H_b7', 'H_b8', 'H_b9']

traindfColNames = ['runs', 'p',
                   'b1_ops', 'b2_ops', 'b3_ops', 'b4_ops', 'b5_ops',
                   'b6_ops', 'b7_ops', 'b8_ops', 'b9_ops']
train_df = pd.DataFrame(columns=traindfColNames)

In [0]:
df = pd.read_csv("/content/drive/My Drive/2018_schedule.csv")

In [0]:
cnt = 0
totalGameDays = df.shape[0]
for index, row in df.iterrows():
    print("{}/{}".format(index+1, totalGameDays))
    rowList = []
    rowList.append(row['A_runs'])
    rowList.append(getRecentWHIP(row['Date'], row['H_sp']))
    for batter in awayLineup:
        #print(row['Date'], row[batter])
        rowList.append(getRecentOPS(row['Date'], row[batter]))
    train_df.loc[cnt] = rowList
    cnt += 1
    #print("")

    rowList = []
    rowList.append(row['H_runs'])
    rowList.append(getRecentWHIP(row['Date'], row['H_sp']))
    for batter in homeLineup:
        #print(row['Date'], row[batter])
        rowList.append(getRecentOPS(row['Date'], row[batter]))
    train_df.loc[cnt] = rowList
    cnt += 1
    #print("")

In [95]:
train_df.tail(50)

,runs,p,b1_ops,b2_ops,b3_ops,b4_ops,b5_ops,b6_ops,b7_ops,b8_ops,b9_ops
4812,5,1.132,0.457,0.51,0.83,0.666,0.65,0.593,0.701,0.929,NaN
4813,6,1.132,0.713,1.982,1.887,0.845,0.258,1.067,0.481,0.6,0.343
4814,3,1.603,0.9,0.567,0.376,0.851,0.755,0.287,0.742,0.599,0.204
4815,8,1.603,0.768,0.93,1.156,0.975,0.795,1.106,0.458,1.8,0.983
4816,0,0.997,0.888,1.357,0.772,0.812,0.491,0.917,0.5,1.052,0
4817,1,0.997,0.58,0.803,0,1.375,1.136,0.848,0.2,0.6,1.273
4818,0,0.991,0.76,1.022,1.057,0.317,0.956,0.809,0.865,0.44,0.142
4819,3,0.991,0.574,0.597,0.733,0.877,0.536,1.015,0.988,1.423,0
4820,5,1.624,0.631,0.809,1.283,1.101,0.235,0.795,0.4,0.707,0
4821,4,1.624,1.271,0.665,0.553,0.979,0.606,0.429,0.3,0.125,0


In [0]:
train_df.to_csv("train.csv", index=False)

In [194]:
train_df = pd.read_csv("train.csv")
train_df.describe()

,runs,p,b1_ops,b2_ops,b3_ops,b4_ops,b5_ops,b6_ops,b7_ops,b8_ops,b9_ops
count,4862.000000,4862.000000,4862.000000,4861.000000,4861.000000,4862.000000,4861.000000,4858.000000,4859.000000,4853.000000,4830.000000
mean,4.448787,1.325072,0.848091,0.886337,0.910307,0.883491,0.845697,0.805727,0.747598,0.677583,0.499314
std,3.192795,0.670156,0.324691,0.336519,0.344250,0.337814,0.341955,0.344591,0.341903,0.343230,0.382908
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.096250,0.633000,0.661000,0.670000,0.650000,0.609000,0.572000,0.521000,0.450000,0.182000
50%,4.000000,1.288000,0.817000,0.862000,0.883000,0.868000,0.833000,0.786000,0.725000,0.653000,0.467000
75%,6.000000,1.502750,1.043000,1.086000,1.118000,1.087000,1.057000,1.015000,0.964000,0.891000,0.750000
max,25.000000,25.000000,2.669000,3.000000,3.300000,4.550000,2.857000,4.000000,3.100000,2.550000,3.000000


In [195]:
train_df = train_df.fillna(0)
train_df = train_df.applymap(lambda x: round(x, 3))
train_df.describe()

,runs,p,b1_ops,b2_ops,b3_ops,b4_ops,b5_ops,b6_ops,b7_ops,b8_ops,b9_ops
count,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000,4862.000000
mean,4.448787,1.325072,0.848091,0.886155,0.910120,0.883491,0.845523,0.805064,0.747137,0.676329,0.496028
std,3.192795,0.670156,0.324691,0.336724,0.344462,0.337814,0.342135,0.345223,0.342301,0.344147,0.383775
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.096250,0.633000,0.661000,0.670000,0.650000,0.609000,0.572000,0.520000,0.450000,0.182000
50%,4.000000,1.288000,0.817000,0.862000,0.883000,0.868000,0.833000,0.786000,0.725000,0.652000,0.463000
75%,6.000000,1.502750,1.043000,1.086000,1.118000,1.087000,1.057000,1.015000,0.964000,0.891000,0.750000
max,25.000000,25.000000,2.669000,3.000000,3.300000,4.550000,2.857000,4.000000,3.100000,2.550000,3.000000


In [196]:
train_df.isnull().any().any()

False

In [197]:
train_df.isnull().values.sum()

0

In [198]:
train_df.isnull().sum().sum()

0

In [199]:
train_df.isnull().values.any()

False

In [0]:
#train_df.to_csv("train_fillna.csv", index=False)

In [0]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(train_df['runs'])
# Remove the labels from the features
# axis 1 refers to the columns
train_df= train_df.drop('runs', axis = 1)
# Saving feature names for later use
feature_list = list(train_df.columns)
# Convert to numpy array
features = np.array(train_df)

In [201]:
features

array([[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       ...,
       [1.12 , 0.898, 0.59 , ..., 0.701, 0.334, 0.494],
       [0.851, 1.414, 0.447, ..., 1.066, 0.316, 0.428],
       [0.851, 0.721, 0.859, ..., 0.643, 1.403, 0.526]])

In [202]:
labels

array([2., 8., 5., ..., 1., 2., 5.])

In [0]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [204]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (3646, 10)
Training Labels Shape: (3646,)
Testing Features Shape: (1216, 10)
Testing Labels Shape: (1216,)


In [205]:
# The baseline predictions are the historical averages
baseline_preds = np.full(test_labels.shape, round(np.average(test_labels), 1))
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  2.45


In [206]:
np.any(np.isnan(train_features))

False

In [207]:
np.all(np.isfinite(train_features))

True

In [208]:
np.any(np.isnan(train_labels))

False

In [209]:
np.all(np.isfinite(train_labels))

True

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, max_depth = 3)
# Train the model on training data
rf.fit(train_features, train_labels);

In [222]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

TypeError: ignored

In [221]:
for i in predictions:
    print(round(i), end=' ')

4.0 5.0 4.0 5.0 4.0 5.0 5.0 5.0 4.0 5.0 4.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 5.0 5.0 5.0 5.0 5.0 5.0 4.0 5.0 4.0 5.0 4.0 4.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 4.0 5.0 4.0 4.0 4.0 5.0 5.0 5.0 4.0 5.0 4.0 4.0 4.0 4.0 5.0 4.0 5.0 5.0 4.0 4.0 5.0 4.0 5.0 5.0 5.0 4.0 4.0 5.0 5.0 5.0 5.0 5.0 5.0 4.0 5.0 4.0 4.0 4.0 4.0 5.0 5.0 4.0 4.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 4.0 5.0 4.0 4.0 4.0 4.0 4.0 5.0 5.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 5.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 5.0 4.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 5.0 5.0 4.0 5.0 5.0 5.0 4.0 5.0 4.0 5.0 5.0 5.0 4.0 5.0 4.0 5.0 4.0 4.0 4.0 4.0 5.0 5.0 4.0 5.0 5.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 4.0 4.0 5.0 4.0 4.0 5.0 4.0 5.0 4.0 5.0 5.0 4.0 5.0 4.0 4.0 4.0 4.0 4.0 5.0 4.0 5.0 5.0 5.0 4.0 4.0 4.0 5.0 4.0 4.0 5.0 5.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0 5.0 5.0 5.0 4.0 5.0 5.0 5.0 4.0 5.0 4.0 4.0 4.0 5.0 4.0 4.0 4.0 4.0 

In [215]:
for i in test_labels:
    print(i, end=' ')

1.0 6.0 1.0 7.0 6.0 14.0 7.0 7.0 1.0 3.0 5.0 4.0 15.0 5.0 8.0 2.0 4.0 2.0 6.0 14.0 2.0 6.0 2.0 3.0 2.0 2.0 6.0 4.0 6.0 5.0 0.0 2.0 6.0 1.0 3.0 2.0 7.0 5.0 3.0 5.0 3.0 5.0 6.0 8.0 3.0 2.0 3.0 2.0 1.0 3.0 4.0 3.0 8.0 0.0 1.0 7.0 6.0 5.0 4.0 3.0 3.0 4.0 5.0 4.0 4.0 9.0 4.0 0.0 9.0 1.0 2.0 3.0 0.0 7.0 6.0 4.0 1.0 1.0 6.0 6.0 1.0 3.0 5.0 4.0 3.0 1.0 4.0 1.0 10.0 2.0 2.0 4.0 2.0 2.0 0.0 9.0 3.0 2.0 3.0 17.0 3.0 5.0 7.0 8.0 0.0 1.0 5.0 5.0 1.0 4.0 1.0 3.0 19.0 9.0 3.0 6.0 4.0 5.0 1.0 3.0 2.0 3.0 1.0 4.0 2.0 2.0 9.0 5.0 0.0 5.0 3.0 8.0 5.0 5.0 6.0 4.0 11.0 3.0 14.0 4.0 4.0 1.0 1.0 3.0 9.0 2.0 1.0 7.0 5.0 6.0 10.0 11.0 8.0 3.0 3.0 3.0 3.0 1.0 4.0 5.0 5.0 1.0 4.0 5.0 15.0 2.0 1.0 14.0 2.0 1.0 4.0 7.0 6.0 6.0 7.0 1.0 1.0 5.0 4.0 3.0 7.0 19.0 0.0 5.0 10.0 3.0 4.0 6.0 8.0 1.0 2.0 5.0 6.0 8.0 0.0 2.0 0.0 7.0 2.0 3.0 7.0 5.0 4.0 3.0 6.0 4.0 12.0 4.0 4.0 7.0 5.0 9.0 5.0 8.0 6.0 10.0 1.0 5.0 13.0 7.0 3.0 2.0 3.0 1.0 8.0 2.0 11.0 10.0 2.0 7.0 8.0 5.0 0.0 1.0 5.0 5.0 5.0 10.0 3.0 6.0 4.0 0.0 12.0 1.0 7.0

In [223]:
for i in errors:
    print(i, end=' ')

3.20817274364079 1.232732923636588 3.124866687715123 2.3681427692930894 1.621593941014992 9.04622871567934 2.4268947656267423 2.223285801174482 3.451022418295697 1.6110768285619361 0.6908431196216744 0.3348237447067195 10.236424472068883 0.09571290901982099 3.146971826665082 2.4234783531421495 0.2934264064714105 2.25484487188624 1.5389349945742747 9.569132787967847 2.0921179397525345 1.2902059181957402 2.667789413985089 1.8170980947434296 2.7966582897947463 2.605457048518689 1.327702823615759 0.6800026360588127 1.6326842025229817 0.42970949818595994 4.499916231881361 2.632007401155695 1.6103941670676463 3.150098710498364 1.5865779460746188 2.098177271371302 2.566357584116581 0.7908260626188754 0.9557201202892109 0.5060602853454519 1.2587802948889442 0.5880561512612763 1.4297231963415697 3.714032437233853 1.3719708779901563 2.234358353442394 1.4339188901783233 2.40465185077702 3.402657871536018 1.737008381507506 0.43029944558775135 1.1108064231863075 3.4632503164636486 4.016975598059401

In [224]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: -inf %.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
